In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pickle
import pandas as pd
import numpy as np

In [4]:
##load the ann trained model, scaler pickle and onehot
model=tf.keras.models.load_model('model.h5')

###load the scaler pickle and encoder
    
with open('onehot_encoder_geo.pkl','rb') as f:
    label_encoder_geo=pickle.load(f)

with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender=pickle.load(f)

with open('scaler.pkl','rb') as f:
    scaler=pickle.load(f)       

In [6]:
#Example input data
input_data={
    'CrediScore':600,
    'Geography':'France',       
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [9]:
##One hot encode 'Geography'
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))


d:\Python3.10\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [10]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_df=pd.DataFrame([input_data])
input_df

,CrediScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [13]:
##Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CrediScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [14]:
##concat encoded columns with input_df and drop 'Geography' column
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CrediScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
## Fix column name to match scaler's expected features
input_df = input_df.rename(columns={"CrediScore": "CreditScore"})

##Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
##Prediction
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 117ms/step


array([[0.04079238]], dtype=float32)

In [18]:
prediction_proba=prediction[0][0]

In [19]:
prediction_proba

0.040792376

In [20]:
if prediction_proba>0.5:
    print(f"Customer is likely to exit with a probability of {prediction_proba}")       
else:
    print(f"Customer is likely to stay with a probability of {1-prediction_proba}")    

Customer is likely to stay with a probability of 0.9592076241970062
